## Reporte Practica 2: Procesamiento de datos <br />Carlos Espinoza - B92786

## <strong>Que son los`Outliers`</strong>

Un outlier es una obervación que se distingue mucho de otras observaciones. Generalmente se definen como obervaciones que están excepcionalmente lejos de la línea de observaciones dentro de los datos.

No hay una forma precisa de detectar e identificar los outliers, pues estos son muy específicos del dataset con el que estemos trabajando.

### Probar el dataset

Para esta prueba se generarán 10K números aleatorios bajo ciertas condiciones. Se va a utilizar una semilla para asegurarnos que siempre vamos a obtener los mismos datos.

In [1]:
# generate gaussian data
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))

mean=50.049 stdv=4.994


### Método de la desviación estándar

La distribución gausseana tiene la propiedad de que la desviación de la media puede usarse para obtener un resumen confiable de la muestra. Por ejemplo con una desviación estándar de la media se puede cubrir un 68% de los datos

In [2]:
# identify outliers with standard deviation
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Identified outliers: 29
Non-outlier observations: 9971


### Método de rango intercuartílico

No todos los datos se encuentran normalizados o lo suficientemente normalizados para ser tratados como una distribución gausseana. Una buena medida estadística para medir las distribuciones en estos tipos de muetras es el rango intercuartílico.

Este rango se calcula mediante la diferencia de los percentiles 75 y 25. El rango define el 50% del medio de los datos, o lo que también se llama `el cuerpo de los datos`.

Para hacer estos cálculos podemos hacer uso del módulo Numpy

In [3]:
# identify outliers with interquartile range
from numpy.random import seed
from numpy.random import randn
from numpy import percentile
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Percentiles: 25th=46.685, 75th=53.359, IQR=6.674
Identified outliers: 81
Non-outlier observations: 9919


### Detección automática de Outliers

En aprendizaje de máquina se utiliza `one-class classification` para atacar el problema de la detección de outliers.

`One-class classification` acomoda un modelo en los datos 'normalizados' y predice si las nuevas entradas son normales, outliers o anomalías.

Este clasificador se base en un dataset de entrenamiento que contiene únicamente datos normalizados, una vez que este esté cargado en el modelo es que se pueden tomar decisiones respecto a las nuevas entradas. 

Se probará con datos de `Boston Housing`

In [4]:
# load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(506, 13) (506,)
(339, 13) (167, 13) (339,) (167,)


In [5]:
# evaluate model on the raw dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 3.417


In [6]:
# evaluate model on training dataset with outliers removed
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(339, 13) (339,)
(305, 13) (305,)
MAE: 3.356


## <strong>Comentarios sobre el ejercicio</strong>

En este ejercicio se cubrió el concepto de 	`outlier`, el cual hace referencia a aquellos datos que se encuentran por fuera del rango o grupo de observaciones normales para un conjunto de datos. Esto lleva a la necesidad de utilizar técnicas estadísticas para tratar con datos tanto normalizados como los que necesitan ser pasados por este proceso. 

### Técnicas utilizadas

1. Método de la desviación estándar
	- Permite encontrar los outliers por medio del análisis de la desviación estándar de la media, aunque para que este método funcione debe utilizarse un conjunto con un comportamiento similar al de la distribución gausseana.

<br>

2. Rango intercuartílico
	- Mediante la diferencia de los percentiles 75 y 25, se puede encontrar cuál es el cuerpo del conjunto de datos, pues este rango hace referencia al 50% o la mitad del conjunto. Los outliers son aquellos que caen afuera de los rangos establecidos.

<br>

3. Detección automática de outliers
	- Utiliza algoritmos de aprendizaje de máquina en los cuales se alimenta un modelo con un set de datos previamente clasificados como 'normales'. Al tener estos datos de entrenamiento, se pueden clasificar las nuevas entradas como normales, outliers o anomalías.


Al igual que en las prácticas anteriores, este proceso es necesario para iniciar el análisis de los datos. Como hemos hablado en clase, es muy probable que el proceso de análisis termine dando un resultado, pero la calidad, veracidad y utilidad de este dependerá tanto de una buena elección de métodos para su tratamiento y preparación como para su análisis. Me imagino que técnicas como esta se utilizan mucho en las grandes empresas que cuentan con almacenes de datos y necesitan hacer un análisis para presentarlo a la gerencia y que sea de utilidad para la toma de decisiones.